In [1]:
import sys
print(sys.version)

3.9.12 (main, Jun  1 2022, 11:38:51) 
[GCC 7.5.0]


In [2]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
tf.keras.backend.set_floatx('float64')

TensorFlow version: 2.8.2


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import os
import csv
import random

import cv2
import imutils
import glob
# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [4]:
#Określenie ścieżek do plików
folder = '../../Dane/Baza_zdjec_zlaczone_1/'
nazwa_CSV = 'nowe.csv'
slesh = '/'
png = '.png'

In [5]:
#Wczytanie zdjęć do tablicy images
images = []
size = 3600

for i in range(size):
    im = Image.open(folder+str(i)+png)
    arr_im = np.array(im) 
    images.append(arr_im) 
print(len(images))

3600


In [6]:
#Wczytanie pliku CSV do tablicy images

with open(folder+nazwa_CSV,"r") as csvfile:
    lines = csvfile.readlines()
csvfile.close()

new = []

for i in lines:
    new.append(i.split(" "))

    
value = []
for i in new[0]:
    value.append(int(i))    
    
print(len(value))

3600


In [7]:
print(value[1799])

1


In [8]:
arr_1 = [0,1,2,3,4,5,6,7,8,9]
arr_2 = [10,11,12,13,14,15,16,17,18,19]

In [9]:
len(images)

3600

In [18]:
#Inicjalizacja tablic
rand_list = []
rand_list = np.zeros((len(images)), dtype=np.uint8)

x_test = []
y_test = []

x_train = []
y_train = []


#Okreslanie wielkosci zbiorów
eighty = int(len(images)*80/100)
twenty = len(images) - eighty

#Losowanie zbioru uczącego
for i in range(eighty):
    
    #Pierwsze losowanie wartości
    rand = random.randint(0,len(images)-1)
    
    #Sprawdzenie czy wylosowana wartość się powtarza
    while (rand_list[rand] == 1) :
        #Jeżeli tak to ponownie losuje
        rand = random.randint(0,len(images)-1)
    
    #Zapisanie zdjecia i wartości do nowej tablicy
    x_test.append(images[rand])
    y_test.append(value[rand])
    rand_list[rand] = 1

#Losowanie zbioru treningowego
for j in range(twenty):
    
    #Pierwsze losowanie wartości
    rand = random.randint(0,len(images)-1)
    
    #Sprawdzenie czy wylosowana wartość się powtarza
    while (rand_list[rand] == 1) :
        #Jeżeli tak to ponownie losuje
        rand = random.randint(0,len(images)-1)
        
    #Zapisanie zdjecia i wartości do nowej tablicy
    x_train.append(images[rand])
    y_train.append(value[rand])
    rand_list[rand] = 1

In [12]:
                                            #Rozmiar 128x64x1 ?
model = tf.keras.models.Sequential([         
    tf.keras.layers.Conv2D(8,3, input_shape= [64,64,1], activation='relu', padding='same'),
    tf.keras.layers.Conv2D(16,3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(32,3, activation='relu', padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [14]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 64, 64, 8)         80        
                                                                 
 conv2d_7 (Conv2D)           (None, 64, 64, 16)        1168      
                                                                 
 conv2d_8 (Conv2D)           (None, 64, 64, 32)        4640      
                                                                 
 flatten_3 (Flatten)         (None, 131072)            0         
                                                                 
 dense_6 (Dense)             (None, 8)                 1048584   
                                                                 
 dense_7 (Dense)             (None, 1)                 9         
                                                                 
Total params: 1,054,481
Trainable params: 1,054,481
No

64
1
